In [2]:
import numpy as np
import pandas as pd
import  re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.SeqUtils.MeltingTemp import make_table, DNA_NN2
import pyperclip
import DNA_tools as dt
#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
Breslauer= mt.DNA_NN1
Santalucia= mt.DNA_NN4 #same as benchling
Tm_Opt =Santalucia

## This notebook intends to automatically design primers with overhangs for USER cloning

In [4]:
#Design User overhang
HM='A'
for i in range(7):
    r=np.random.rand(1)
    if r <=0.625:
        HM+=np.random.choice(['G','C'])
    else:
        HM+=np.random.choice(['A','T'])
HM+='U'
print(HM, '  GC%:',(HM.count('C')+HM.count('G'))/len(HM))



Tm=0
for bp in HM:
    if bp == 'A' or bp == 'T':
        Tm+=2
    else:
        Tm+=4
Rv=''
for bp in HM[::-1]:
    if bp == 'A':
        Rv+='T'
    if bp == 'T' or bp == 'U':
        Rv+='A'
    if bp == 'C':
        Rv+='G'
    if bp == 'G':
        Rv+='C'
        
        
print('TM:',Tm, len(HM))
print('Reverse: ', Rv)

AATCGACCU   GC%: 0.4444444444444444
TM: 28 9
Reverse:  AGGTCGATT


In [4]:
#Design 50bp random overhang for yeast HR:


continue_search = True
while continue_search == True:
    Overhang=''
    for i in range(50):
        r=np.random.rand(1)
        
        bp=''
        if r<= 0.25:
            bp='A'
        elif r>0.25 and r<=0.5:
            bp='T'
        elif r>0.5 and r<=0.75:
            bp='C'
        else:
            bp='G'
        Overhang+=bp
    
    GC =(Overhang.count('C')+Overhang.count('G'))/len(Overhang)
    if GC >= 0.475 and GC <= 0.525: #makes sure the overhang is balanced
        continue_search = False
        
Rv_Overhang= reverse_complement(Overhang)
print('GC content:', GC, '\n','Overhang Fwd: ', Overhang,'\n Overhang Rv: ', Rv_Overhang)

GC content: 0.5 
 Overhang Fwd:  ATTGGACTACTTTAGCGGCTTCGCGACCAGGCTTTGCCTTAACGGTGATT 
 Overhang Rv:  AATCACCGTTAAGGCAAAGCCTGGTCGCGAAGCCGCTAAAGTAGTCCAAT


### Define your user overhangs:

In [5]:
#forward seq of user overhangs:
User_Overhangs={}
User_Overhangs['GV1']= 'CGTGCGAu'
User_Overhangs['GV2']= 'ATCGCGTG'#'CACGCGAu'
User_Overhangs['U1'] = 'AGCCCTCTu'
User_Overhangs['U2'] = 'ACACAGGCu'
User_Overhangs['U3'] = 'ATGCCACCu'
User_Overhangs['U4'] = 'AAGGAGCGu'
User_Overhangs['U5'] = 'ACCGTAGCu'
User_Overhangs['U6'] = 'ACTGCGAGu'
User_Overhangs['U7'] = 'AGTGCAGGu'
User_Overhangs['none'] = 'TATATATATATATA'

Copy and paste sequence from benchling 
The primers will be designed taking the immediate bp at the start and end of sequence 
(bp are added to annealling sequence till Tm reaches specified Tm (or avg Tm if speficied primers))
Always copy sequence as is (not reverse complement). 
If the seq needs to be reversed in assembly set partX['need_reverse']=1
If you provide primers, input annealing sequence from 5'>>3'
Specify the user sites as wanted in the final construct

In [6]:
#Imput your parts here
part1={}
part1['name']='pMeLS0025(pCYC_benO::yEGFP::Cyc1t)'
part1['seq']='gatccaggcaactttagtgctgacacataatactccataggtattttattatacaaataatgtgtttgaacttattaaaacattcttttaaggtataaacaacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaataatctgtcataaaacaatgtctaaaggtgaagaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaaggtgatgctacttacggtaaattgaccttaaaatttatttgtactactggtaaattgccagttccatggccaaccttagtcactactttcggttatggtgttcaatgttttgcgagatacccagatcatatgaaacaacatgactttttcaagtctgccatgccagaaggttatgttcaagaaagaactatttttttcaaagatgacggtaactacaagaccagagctgaagtcaagtttgaaggtgataccttagttaatagaatcgaattaaaaggtattgattttaaagaagatggtaacattttaggtcacaaattggaatacaactataactctcacaatgtttacatcatggctgacaaacaaaagaatggtatcaaagttaacttcaaaattagacacaacattgaagatggttctgttcaattagctgaccattatcaacaaaatactccaattggtgatggtccagtcttgttaccagacaaccattacttatccactcaatctgccttatccaaagatccaaacgaaaagagagaccacatggtcttgttagaatttgttactgctgctggtattacccatggtatggatgaattgtacaaataaatcgcgtgcattcatccgctctaaccgaaaaggaaggagttagacaacctgaagtctaggtccctatttatttttttatagttatgttagtattaagaacgttatttatatttcaaatttttcttttttttctgtacagacgcgtgtacgcatgtaacattatactgaaaaccttgcttgagaaggttttgggacgctcgaagatcgcgtcagctgaa' #sequence to clone
part1['need_reverse']=0 #if parts needs to be reversed:1 else:0
part1['Primer_up_U']='GV1' #user site up
part1['Primer_down_U']='GV2' #user site down
part1['Primer_up']=''
part1['Primer_up_Tm']=''
part1['Primer_down']=''
part1['Primer_down_Tm']=''

part2={}
part2['name']='pMeLS0076(Rev1p::BenM(H110R, F211V, Y286N))'
part2['seq']='gaatgcggccgctttaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaagagctccagcttttgttccctttagtgagggttaattgcgcgcttggcgtaatcatggtcatagctgtttcctgtgtgaaattgttatccgct'
part2['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2['Primer_up_U']='GV1' #user site up
part2['Primer_down_U']='GV2' #user site down
part2['Primer_up']=''
part2['Primer_up_Tm']=''
part2['Primer_down']=''
part2['Primer_down_Tm']=''

part2B={}
part2B['name']='pMeLS0076(Rev1p::BenM_MP17_D08)'
part2B['seq']='Ttaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaa'
part2B['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2B['Primer_up_U']='GV1' #user site up
part2B['Primer_down_U']='GV2' #user site down
part2B['Primer_up']=''
part2B['Primer_up_Tm']=''
part2B['Primer_down']=''
part2B['Primer_down_Tm']=''


part3={}
part3['name']='pTDH2'
part3['seq']='tttgttttgtttgtttgtgtgatgaatttaatttgaaactaagttcttggtttttttaacatttaaacaaacaataataaaatgaatagacacacaagaaaaaaactatcttgaatgagtgcaacctttatatattcttccaaacctacaaaaaaattgcaggtctgcgatcactgggtactatcaaaacacatattagcatacgtatgtgttcgtacgtgcggtccgcaaggtctttgtgacagccatgacatgattggattgaatagttcacttcagctgtcaccaccagttttaggccctaacacatcagtcgagttccactaggctttgttagccgtagctgcgttgcaacaactgcgcaaaacgcgcgacttcctgtaaatgccgcagatcacgtggtaaagaaaagaaacaacaggccgacccagtttcgttgtttcttttcattatttctggatgctcttcacaacgtatttcatgacatacttattgaattagtggaagcccaaaacatcattaccctggaaaacttaattgacgtaaattaacttcagttgtcttatgttttttgcatccttaactttagccgcgcagggataatcatatatgtcgaaagagcaactaaacaatacttaaaaacgtgtttgaacttttacgaaagggtaacgacaaataatcattcactggaaaaaccaataaaccataagtaaatgaaaaactgtttcaaccgatgttggtactttccatgtcttaaaagagtacaatttataaattaatgaaaacacagaagtatttagatcggctcaaatgtttttggacattaaaagatcttgaaactgagtaagatgctcagaatacccgtcaagataagagtataatgtagagtaatataccaagtattcagcatattctcctcttcttttgtataaatcacggaagggatgatttataagaaaaatgaatactattacacttcatttaccaccctctgatctagat'
part3['need_reverse']=1 #if parts needs to be reversed:1 else:0
part3['Primer_up_U']='GV1' #user site up
part3['Primer_down_U']='U1' #user site down
part3['Primer_up']=''
part3['Primer_up_Tm']=''
part3['Primer_down']='aaatctagatcagagggtggtaaat'
part3['Primer_down_Tm']=''

part4={}
part4['name']='pCFB1237_ScTkl1'
part4['seq']='tcagaaagcttttttcaaaggagaaattagcttgtcacccttatagaatgcaatggtcttttgagctctttcagcaacaccttctggggtgaaaccgaagaacttgaagacttctggtgccttaccggaggcaccaaatctgtcaataccgaaggattgatgagcgtatttgccccaacatgtggtagccaaaacttcaacagacatgattggaacgttgtctggtaagactgatagtctgtattctaggggttgtttgtcaaaagtgaagaaatctggtagagaaacaacacgagccttgatgttctttgcggccaaagtcttagcagcttcaacactcaaagacacttcggaaccagtagccactaaaataatatctgggttagcaacatcttgtagtacgtaaccacccttagaagcgctttcaatagagctaccttccaattgtggcaagttttgtctggacaaagcaatgatacttggagtatgcttggattctaaagagttcttgtaggcggcagaaacttcgttaccatcagctggtctccaaacttgaatgtttggtagggatctgaagtgtgctaaagtttcaataggttgatgtgttggaccatcttcaccgacaccgatagagtcatgtgtagcaacccaaataactgggtggccagacaaagcggacaatctaacggcaccagcagcataagaaacgaagttcaagaaagtaccaccgtatggtttgtagttggcaccgaaagctgaaataccgttcattatggcacccatagcgtgttctctaataccgtacctaatgtatctaccagagtagttacctgaaccggaagaaggaggttggaagtcaagggcttccttccatctggtcaagttagaaggtgttaaatcggcagaaccaccaatcaactctggcaattgattgtaaacatcctcaagaacagtttctgataattttctagtggccacggcagagtccttggcggtgtaagttggcaacttagattcccaatttgcgggtagttggccgctcaatcttctagccaattcagcacctaattctgggaatttcttttggtattcgctgaacaacttgttccacttgttgttggcttcgacacctggctttaaaattgtcttttggtagtggtcgtaaacttcttgtggaacaacaaaggacttgtctgggttgaaaccgaatttgctctttagttgtttaacatcatctgctttcaatggggcaccgtgcacagagtgagagccggcatgcaaggaaccgtaaccaatggttgtggtcattttgatcaaagttggtttgtccttggataacttagcttgagcaatagccttggcaataccggctagatcttcgttaccattttctacgtacaaaacttcccaaccgtaggcttcgtatctcttagcaacatcttcatcgaatgagatactggtagcaccatcgatagtgatcttgttgtcatcgtagatggcaatcaagttacccaatttcaaatgaccagccaaggaggaagcttctgaagaaataccttcttgcaaacaaccgtcacccaagaaaacataggtgtagttgtcagacaaggtaaagcccggcttgttgtaagtggcagccaggttagcttgagccatggccataccaacagcgttggagataccttgacctaatggaccggtagtaacttcaacacctggcaactcaaattcaggatgacctggtgttctggaacccaactgtctgaactgtttcaagtcttcaatagacagatcgtaaccagtcaaatgtagcatagaatacaacaaagcgaccgcgtgaccgttagacaagacaaatctatctctgttgatccagtctgggttggttgggttcatgcgcatttgactccatagaacgtgtgcagctggtgccatacccaatggagcacctgggtgacctgagttggccttggatacggtgtccacagccaaaattcttatggtggagacggctagcttatcaatgtcagtgaattgagtcattgt'
part4['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part4['Primer_up_U']='U1' #user site up
part4['Primer_down_U']='U2' #user site down
part4['Primer_up']=''
part4['Primer_up_Tm']=''
part4['Primer_down']=''
part4['Primer_down_Tm']=''
part5={}
part5['name']='ANTE113_Tdh3bidir'
part5['seq']='TCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCAC'
part5['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part5['Primer_up_U']='U2' #user site up
part5['Primer_down_U']='GV2' #user site down
part5['Primer_up']=''
part5['Primer_up_Tm']=''
part5['Primer_down']=''
part5['Primer_down_Tm']=''


User1=[part1]
User2=[part2]
User2B=[part2B] #pCCM010
User3=[part3, part4,part5]

In [4]:
part6={}
part6['name']='FBA1p'
part6['seq']='tttgaatatgtattacttggttatggttatatatgacaaaagaaaaagaagaacagaagaataacgcaaggaagaacaataactgaaattgatagagaagtattatgtctttgtctttttataataaatcaagtgcagaaatccgttagacaacatgagggataaaatttaacgtgggcgaagaagaaggaaaaaagtttttgtgagggcgtaattgaagcgatctgttgattgtagattttttttttttgaggagtcaaagtcagaagagaacagacaaatggtattaaccatccaatacttttttggagcaacgctaagctcatgcttttccattggttacgtgctcagttgttagatatggaaagagaggatgctcacggcagcgtgactccaattgagcccgaaagagaggatgccacgttttcccgacggctgctagaatggaaaaaggaaaaatagaagaatcccattcctatcattatttacgtaatgacccacacatttttgagattttcaactattacgtattacgataatcctgctgtcattatcattattatctatatcgacgtatgcaacgtatgtgaagccaagtaggcaattatttagtactgtcagtattgttattcatttcttagggcttgcgtcagcttgatgaggatttgtgatagtagccgttggaggcatgaagggcattggaaacatggggaaaggaggtataccgaaaggtaacggtactggtggcactggcgtaccgccgttatttacagaagttggaaggctggtattgttgttcaagccagcggtgccagttggatccatcaacttcattttcttagtttctggctggttgccatctttagagcttccgtgtagttcctcttgtttcttcaaaaacgtttcgacctccttttccttgtcctggtcgggtactaaagaatcgagaaggatatcgcgggtttcgcaattggggcatacaaagtcgctctctaccagt'
part6['need_reverse']=1 #if parts needs to be reversed:1 else:0
part6['Primer_up_U']='GV1' #user site up
part6['Primer_down_U']='U1' #user site down
part6['Primer_up']='tttgaatatgtattacttggttatg'
part6['Primer_up_Tm']=''
part6['Primer_down']='actggtagagagcgacttt'
part6['Primer_down_Tm']=''

part7={}
part7['name']='pCFB1239_PaAroZ'
part7['seq']='tcacaaagcagctgacaaagacaacattctgataggttgttgtgtagcttgttgttgagttggagagtcaacggtgatcttcatatctctaaccaatttagcccaagaaacagcacctcttctagctaattcttctggaacaccaaaaccggtatcggacattcttctgttgaataattctaaggaaacccaaccttcaaaacccaaaccgttgaaaaaagcccaagcaatttcttttactggcaagtaagcacctctatccttttcaccgtagaataatctgcagtttctagaccaagacattctagctggttgttctggatcatagaatctgtgacctggaaccaatggtttcttcaacttttcggcatcaacaacttgaacgtagaaaaccttggaaacgtcaactctttcaaccaatctggcaatggactttctgatagcttcttcagcatttggagttctaccagaagcaacagttggatcagcataaactctaccggcaatgttaaaagtgtccaaacaaacaccgaagtttggtctgttaactctttgaacaacttcccaagatctttcccaagtatcaactctagtggaccaacacaaagcttcgtaaacgaatctaattggtgggttagcttgtaaacccatatcagctacttcttgcaagtcggagacaatcaaagagatatcttcagtaacttcttcggctggcaagaagttggctggaatttgaataatatcggtatccaattcatgggccaattcgatccagaattccaattgttccaatcttctttcgtgttcttctctatccaacaaaccatcgtattgagagaatggttgcaagcaaacgatttcgatgtttctgacttggcacattctcaaaatttgtctagcagcagacaattgagcagctggtgatggaccacatggagatggagtttcaccagacaatctataggcgacatcagccaaatcttcatggaataattccaaaccttggtaaccgtactttctagccatatccaacttagtagtgaaagaatgaccggcataacatctacccaaagacatagaggtaatggccaacttggatggcattgt'
part7['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part7['Primer_up_U']='U1' #user site up
part7['Primer_down_U']='U2' #user site down
part7['Primer_up']=''
part7['Primer_up_Tm']=''
part7['Primer_down']=''
part7['Primer_down_Tm']=''

part8={}
part8['name']='His5t'
part8['seq']='atagattaatttaaacagtatatgtacagttttatatatatatatatatatatatacatatataaagaaacctgtgcgttttttgtattttcaaatacatttagttttggcgccacttctataaaaggtctcatgatattgttac'
part8['need_reverse']= 0  #if parts needs to be reversed:1 else:0
part8['Primer_up_U']='U2' #user site up
part8['Primer_down_U']='GV2' #user site down
part8['Primer_up']='atagattaatttaaacagtatatgtacagttt'
part8['Primer_up_Tm']=''
part8['Primer_down']='gtaacaatatcatgagaccttttata'
part8['Primer_down_Tm']=''

part9={}
part9['name']='TPI1p'
part9['seq']='ttttagtttatgtatgtgttttttgtagttatagatttaagcaagaaaagaatacaaacaaaaaattgaaaaagattgatttagaattaaaaagaaaaatatttacgtaagaagggaaaatagtaaatgttgcaagttcactaaactcctaaattatgctgccctttatattccctgttacagcagccgagccaaaggtatataggctcctttgcattagcatgcgtaacaaaccacctgtcagtttcaaccgaggtggtatccgagagaattgtgtgattgctttaattaatttcggagaatctcacatgccactgaagattaaaaactggatgccagaaaaggggtgtccaggtgtaacatcaatagaggaagctgaaaagtcttagaacgggtaatcttccaccaacctgatgggttcctagatataatctcgaagggaataagtagggtgataccgcagaagtgtctgaatgtattaaggtcctcacagtttaaatcccgctcacactaacgtaggattattataactcaaaaaaatggcattattctaagtaagttaaatatccgtaatctttaaacactatgtagttaggtctccctcacaatcagtccatttgggtagcacggtcctcaggacgtatctattgatggattcgtccagttccatcaccattacgctcccgttaggaacattggtaaacgattcaaactcttcgtatgtccatctaaaccatttcatcaggaatactctggaataaataccatgtgtaactaggacaacaacatctctgggtcttctctcttgcctatcatggaagtgcctgaataaagtctcttggaaactggcgactctgtcatatacatctgccgcactttctccatgagggaatctgaagaagaaatgaccatacgtagatctcttcttcataacgtcctgcatgctattgattttttggaaattaccaaaatcttgttcccttattcttggctcatcctt'
part9['need_reverse']=1 #if parts needs to be reversed:1 else:0
part9['Primer_up_U']='GV1' #user site up
part9['Primer_down_U']='U1' #user site down
part9['Primer_up']=''
part9['Primer_up_Tm']=''
part9['Primer_down']=''
part9['Primer_down_Tm']=''

part10={}
part10['name']='pCfB2193_NatMX'
part10['seq']='accatgggtaccactcttgacgacacggcttaccggtaccgcaccagtgtcccgggggacgccgaggccatcgaggcactggatgggtccttcaccaccgacaccgttttccgcgtcaccgccaccggggacggcttcaccctgcgggaggtgccggtggacccgcccctgaccaaggtgttccccgacgacgaatcggacgacgaatcggacgacggggaggacggcgacccggactcccggacgttcgtcgcgtacggggacgacggcgacctggcgggcttcgtggtcgtctcgtactccggctggaaccgccggctgaccgtcgaggacatcgaggtcgccccggagcaccgggggcacggggtcgggcgcgcgttgatggggctcgcgacggagttcgcccgcgagcggggcgccgggcacctctggctggaggtcaccaacgtcaacgcaccggcgatccacgcgtaccggcggatggggttcaccctctgcggcctggacaccgccctgtacgacggcaccgcctcggacggcgagcaggcgctctacatgagcatgccctgcccctaa'
part10['need_reverse']= 0#if parts needs to be reversed:1 else:0
part10['Primer_up_U']='U1' #user site up
part10['Primer_down_U']='U2' #user site down
part10['Primer_up']=''
part10['Primer_up_Tm']=''
part10['Primer_down']=''
part10['Primer_down_Tm']=''

part11={}
part11['name']='LSC2t'
part11['seq']='aaaattaaaaaaaaaaaaaagaaattttttccaaaagagtatatttatatgtatgtatacatgtagggaaaataagaaactttattaatagtaaaaaagcatatatactttattattaactcttttgtttttctcgagaagc'
part11['need_reverse']=1 #if parts needs to be reversed:1 else:0
part11['Primer_up_U']='U2' #user site up
part11['Primer_down_U']='GV2' #user site down
part11['Primer_up']='aaaattaaaaaaaaaaaaaagaaattttttcc'
part11['Primer_up_Tm']=''
part11['Primer_down']='gcttctcgagaaaaacaaaagagtta'
part11['Primer_down_Tm']=''

part12={}
part12['name']='pCYC1'
part12['seq']='cagcattttcaaaggtgtgttcttcgtcagacatgttttagtgtgtgaatgaaataggtgtatgttttctttttgctagacaataattaggaacaaggtaagggaactaaagtgtagaataagattaaaaaagaagaacaagttgaaaaggcaagttgaaatttcaagaaaaaagtcaattgaagtacagtaaattgacctgaatatatctgagttccgacaacaatgagtttaccaaagagaacaatggaataggaaactttgaacgaagaaaggaaagcaggaaaggaaaaaatttttaggctcgagaacaatagggcgaaaaaacaggcaacgaacgaacaatggaaaaacgaaaaaaaaaaaaaaaaacacagaaaagaatgcagaaagatgtcaactgaaaaaaaaaaaggtgaacacaggaaaaaaaataaaaaaaaaaaaaaaaaaaggaggacgaaacaaaaaagtgaaaaaaaatgaaaatttttttggaaaaccaagaaatgaattatatttccgtgtgagacgacatcgtcgaatatgattcagggtaacagtattgatgtaatcaatttcctacctgaatctaaaattcccgggagcaagatcaagatgttttcaccgatctttccggtctctttggccggggtttacggacgatggcagaagaccaaagcgccagttcatttggcgagcgttggttggtggatcaagcccacgcgtaggcaatcctcgagcagatccgccaggcgtgtatatatagcgtggatggccaggcaactttagtgctgacacatacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaata'
part12['need_reverse']=0 #if parts needs to be reversed:1 else:0
part12['Primer_up_U']='GV1' #user site up
part12['Primer_down_U']='U1' #user site down
part12['Primer_up']='cagcattttcaaaggtgt'
part12['Primer_up_Tm']=''
part12['Primer_down']=''
part12['Primer_down_Tm']=''

part13={}
part13['name']='pCFB1239CaCatA'
part13['seq']='acaatgtcccaagctttcaccgaatctgttaagacttctttgggtccaaatgctactccaagagctaaaaagttgattgcctctttggttcaacacgttcatgatttcgctagagaaaaccatttgactaccgaagattggttgtggggtgttgatttcattaacagaattggtcaaatgtccgactccagaagaaacgaaggtattttggtttgcgatatcatcggtttggaaaccttggttgatgctttgactaacgaatccgaacaatctaaccatacctcctctgctattttgggtcctttttacttgccagattctccagtttatccaaacggtggttctatcgttcaaaaggctattccaactgatgttaagtgcttcgttagaggtaaggttactgatactgaaggtaaaccattgggtggtgctcaattggaagtttggcaatgtaattctgctggtttctactctcaacaagctgatcatgatggtccagaattcaatttgagaggtactttcattaccgacgacgaaggtaattactccttcgaatgtttaagaccaacctcctatccaattccatacgatggtcctgctggtgatttgttgaaaatcatggatagacatccaaacagaccatcccatattcattggagagtttctcatccaggttaccatactttgatcacccaaatctatgatgctgaatgtccatacaccaacaacgattctgtttacgctgttaaggatgacatcatcgttcacttcgaaaaggttgataacaaggataaggatttggtcggtaaggtcgaatacaagttggattacgatatttccttggccaccgaatcctctattcaagaagctagagctgctgctaaagctagacaagatgctgaaatcaagttgtaa'
part13['need_reverse']=0 #if parts needs to be reversed:1 else:0
part13['Primer_up_U']='U1' #user site up
part13['Primer_down_U']='U2' #user site down
part13['Primer_up']=''
part13['Primer_up_Tm']=''
part13['Primer_down']=''
part13['Primer_down_Tm']=''

part14={}
part14['name']='CPS1t'
part14['seq']='gcgcaatgattgaatagtcaaagattttttttttttaattttttttttttaattttttttttttttcatagaactttttatttaaataaatcacgtctatatatgtatcagtataacgtaaaaaaaaaaacaccgtcagttaaacaaaacataaataaaaaaaaaaagaagtgtcaaatcaagtgtcaaatc'
part14['need_reverse']=0 #if parts needs to be reversed:1 else:0
part14['Primer_up_U']='U2' #user site up
part14['Primer_down_U']='GV2' #user site down
part14['Primer_up']='gcgcaatgattgaatagtcaaa'
part14['Primer_up_Tm']=''
part14['Primer_down']='gatttgacacttgatttgacacttcttt'
part14['Primer_down_Tm']=''

User4=[part6, part7,part8]
User5=[part9, part10,part11]
User6=[part12, part13,part14]

In [24]:
part15={}
part15['name']='pCfB1241_TDH3p:KpAroY.B'
part15['seq']='tcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part15['need_reverse']=1#if parts needs to be reversed:1 else:0
part15['Primer_up_U']='GV1' #user site up
part15['Primer_down_U']='U2' #user site down
part15['Primer_up']=''
part15['Primer_up_Tm']=''
part15['Primer_down']=''
part15['Primer_down_Tm']=''


part16={}
part16['name']='VPS13t'
part16['seq']='gcgcgctgcggatatttctaaggcatggtcgtgcggagctacaataatacgattgaattatagctacatagtgtacaaaagcgggtatatactttcatatgtga'
part16['need_reverse']=1#if parts needs to be reversed:1 else:0
part16['Primer_up_U']='U2' #user site up
part16['Primer_down_U']='GV2' #user site down
part16['Primer_up']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part16['Primer_up_Tm']=''
part16['Primer_down']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part16['Primer_down_Tm']=''

User7=[part15, part16]


In [9]:
# this user assembly was redesigned to accomodate a gBlock for AroyB P146T


part1={}
part1['name']='pCfB1241_TDH3p'
part1['seq']='tttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part1['need_reverse']=1#if parts needs to be reversed:1 else:0
part1['Primer_up_U']='GV1' #user site up
part1['Primer_down_U']='U7' #user site down
part1['Primer_up']=''
part1['Primer_up_Tm']=''
part1['Primer_down']=''
part1['Primer_down_Tm']=''

part2={}
part2['name']='KpAroY.B-P146T.gBlock'
part2['seq']='aaacaatgaagttgatcatcggtatgactggtgctacaggtgctccattgggtgttgctttgttgcaagctttgagagatatgccagaagttgaaacccatttggttatgtctaaatgggctaagaccaccattgaattggaaactccatggactgctagagaagttgctgctttggctgatttttctcattctccagctgatcaagctgctactatttcttctggttctttcagaactgatggtatgatcgttattccatgctctatgaaaaccttggctggtattagagctggttatgctgaaggtttggttggtagagctgctgatgttgttttgaaagaaggtagaaagttggtcttggtcccaagagaaatgccattgtctactatccatttggaaaacatgttggccttgtctagaatgggtgtagctatggttAcaccaatgccagcttattacaatcatccagaaaccgttgatgacatcaccaaccatatagttaccagagttttggaccaattcggtttggattatcacaaagctagaagatggaacggtttgagaactgctgaacaattcgctcaagaaattgaatga'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part2['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2['Primer_up_U']='U7' #user site up
part2['Primer_down_U']='U2' #user site down
part2['Primer_up']=''
part2['Primer_up_Tm']=''
part2['Primer_down']=''
part2['Primer_down_Tm']=''

part3={}
part3['name']='VPS13t'
part3['seq']='gcgcgctgcggatatttctaaggcatggtcgtgcggagctacaataatacgattgaattatagctacatagtgtacaaaagcgggtatatactttcatatgtga'
part3['need_reverse']=1#if parts needs to be reversed:1 else:0
part3['Primer_up_U']='U2' #user site up
part3['Primer_down_U']='GV2' #user site down
part3['Primer_up']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part3['Primer_up_Tm']=''
part3['Primer_down']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part3['Primer_down_Tm']=''

pCCM011=[part1, part2, part3]

In [6]:

part17={}
part17['name']='pCFB1241_Tef1p_KpAroyC.iso'
part17['seq']='gcacacaccatagcttcaaaatgtttctactccttttttactcttccagattttctcggactccgcgcatcgccgtaccacttcaaaacacccaagcacagcatactaaatttcccctctttcttcctctagggtgtcgttaattacccgtactaaaggtttggaaaagaaaaaagagaccgcctcgtttctttttcttcgtcgaaaaaggcaataaaaatttttatcacgtttctttttcttgaaaatttttttttttgatttttttctctttcgatgacctcccattgatatttaagttaataaacggtcttcaatttctcaagtttcagtttcatttttcttgttctattacaactttttttacttcttgctcattagaaagaaagcatagcaatctaatctaagttttaattacaaatctgtcatatctgtcataaaacaatgaccgccccaatccaagatttgagagatgctattgctttgttacaacaacacgacaatcaatacttggaaaccgatcatccagttgatccaaatgctgaattggctggtgtttacagacatattggtgctggtggtactgtaaaaagaccaactagaattggtccagccatgatgttcaacaacattaagggttatccacactccagaatcttggttggtatgcatgcttctagacaaagagcagctttgttgttgggttgtgaagcttctcaattggctttggaagttggtaaagctgttaagaaaccagttgctccagttgttgttccagcttcttctgctccatgtcaagaacaaattttcttggctgatgatccagacttcgatttgagaactttgttgccagctcataccaacactccaattgatgctggtccatttttttgtttgggtttggctttagcttctgatcctgttgatgcttctttgaccgatgttaccattcatagattgtgcgttcaaggtagagatgaattgtctatgtttttggctgccggtagacatatcgaagtttttagacaaaaagctgaagctgctggtaagccattgccaattactattaacatgggtttagatccagccatctacattggtgcttgttttgaagctccaactactccatttggttacaacgaattgggtgttgctggtgctttgagacaaagaccagttgaattggttcaaggtgtttctgttccagaaaaggctattgctagagccgaaatagttatcgaaggtgaattattgccaggtgtcagagttagagaagatcaacatacaaattccggtcatgctatgccagaatttccaggttattgtggtggtgctaatccatctttgccagttattaaggttaaggccgttaccatgagaaacaacgctattttacaaactttggtcggtccaggtgaagaacatacaactttggctggtttgccaaccgaagcttctatttggaatgctgttgaagctgcaattccaggtttcttgcaaaatgtttatgctcatacagctggtggtggtaagttcttgggtatattgcaagtcaagaaaagacaaccagctgacgaaggtagacaaggtcaagctgctttattagctttggctacttactccgaattgaagaatatcatcttggtcgatgaagatgttgatatcttcgattccgatgatattttgtgggctatgactactagaatgcaaggtgatgtttccattactaccattccaggtattagaggtcaccaattagatccatctcaaaccccagaatactccccatcaattagaggtaatggtatctcctgtaagaccattttcgattgcactgttccatgggctttgaagtctcattttgaaagagcaccatttgctgacgttgatcctagaccttttgctccagaatatttcgctagattggaaaagaatcaaggttccgctaagtaa'
part17['need_reverse']=0 #if parts needs to be reversed:1 else:0
part17['Primer_up_U']='GV1' #user site up
part17['Primer_down_U']='U2' #user site down
part17['Primer_up']=''
part17['Primer_up_Tm']=''
part17['Primer_down']=''
part17['Primer_down_Tm']=''



part18={}
part18['name']='IDP1t'
part18['seq']='tcgaatttacgtagcccaatctaccacttttttttttcattttttaaagtgttatacttagttatgctctaggataatgaactacttttttttttttttttttactgttatcataaatatatataccttattgttgtttgcaaccgtcggttaattccttatcaaggttccccaagttcggatcattaccatc'
part18['need_reverse']=0 #if parts needs to be reversed:1 else:0
part18['Primer_up_U']='U2' #user site up
part18['Primer_down_U']='GV2' #user site down
part18['Primer_up']='tcgaatttacgtagcccaatcta'
part18['Primer_up_Tm']=''
part18['Primer_down']='gatggtaatgatccgaacttgg'
part18['Primer_down_Tm']=''
User8=[part17, part18]

In [7]:
part19={}
part19['name']='CCW12p'
part19['seq']='tattgatatagtgtttaagcgaatgacagaagattaatttcttggtatgttaggaaagaataaaggagaataagaataattagaacaatgtaggatggaaagaaagattatcaagcatgccgactttatatacttgaacggaggcaaaggatgcaaaattttctcacatttctttctgccgttatgttggaagtaagactcccattatcgcaatactgcaacacgaatatgcaaaatttgctgagttatcgcagatagttgttgcaaagatagcggcgtaggtggccgcgaaatggggaattccaaaacaaacggtttttttactcctgagaaatacttgtacgggataatccagggcctaccacccacgcttcgaggattggcttttatttttttttttttggtggcgttttatttctttcccgctttctgggacttgtgcggagttttgagaggggcgcgcggcaaaggattcccaaaacggaaatcagacgccaatagccagcactcaaagcagttctggacccattccgattttcccatttggttcttgcgcgtgctgattccgacacgcgcgtctataaatagcatgaagtatccgcacaccgcagcgttagtgaggtgagggtggcagcaagctaattcccgcatctggaatctgaactgccccttttggactaaccgtgtggttcatgggtgggcgaagtgcgcaacctcgaaggttttcttttgcgtgtcggattttacatccggcggtagcgcatgatgccatggctggctccagatacatcctcagggcaccagcatctataattagattggcgcaacatggctggctgcactgctgtcttcacttctttctttttccggcaatgaatgatgtatgttttgtggcaaaagggtccgcattgtacctgtttacagttgagattatcgtttttggtagcccttcattacggcataacgtattaagtttcttt'
part19['need_reverse']=1 #if parts needs to be reversed:1 else:0
part19['Primer_up_U']='GV1' #user site up
part19['Primer_down_U']='U1' #user site down
part19['Primer_up']=''
part19['Primer_up_Tm']=''
part19['Primer_down']='aaagaaacttaatacgttatgcc'
part19['Primer_down_Tm']=''

part20={}
part20['name']='pCfB1237_KpAroY.D'
part20['seq']='acaatgatctgtccaagatgcgccgacgaaaaaattgaagttatggctacttctccagttaagggtgtttggactgtttatcaatgtcaacactgcttgtacacttggagagatactgaaccattgagaagaacctctagagaacattaccctgaagctttcagaatgacccaaaaggatattgatgaagctccacaagttcctcatgttccaccattattgccagaagataagagataa'
part20['need_reverse']=0 #if parts needs to be reversed:1 else:0
part20['Primer_up_U']='U1' #user site up
part20['Primer_down_U']='U2' #user site down
part20['Primer_up']=''
part20['Primer_up_Tm']=''
part20['Primer_down']=''
part20['Primer_down_Tm']=''

part21={}
part21['name']='PRM9t'
part21['seq']='acagaagacgggagacactagcacacaactttaccaggcaaggtatttgacgctagcatgtgtccaattcagtgtcatttatgattttttgtagtaggatataaatatatacagcgctccaaatagtgcggttgccccaaaaacaccacggaacctcatctgttctcgtactttgttgtgacaaagtagctcactgccttattatcacattttcattatgcaacgcttcggaaaatacgatgttgaaaat'
part21['need_reverse']=0 #if parts needs to be reversed:1 else:0
part21['Primer_up_U']='U2' #user site up
part21['Primer_down_U']='GV2' #user site down
part21['Primer_up']='' #we dont provide Curran's primer because their TM is very high for PRM9t
part21['Primer_up_Tm']=''
part21['Primer_down']='' #we dont provide Curran's primer because their TM is very high for PRM9t
part21['Primer_down_Tm']=''
User9=[part19, part20, part21]

In [29]:
CCM004={}
CCM004['name']='pCCM004(pFBA1:PaAroZ:tHis5)'
CCM004['seq']='ACTGGTAGAGAGCGACTTTgtatgccccaattgcgaaacccgcgatatccttctcgattctttagtacccgaccaggacaaggaaaaggaggtcgaaacgtttttgaagaaacaagaggaactacacggaagctctaaagatggcaaccagccagaaactaagaaaatgaagttgatggatccaactggcaccgctggcttgaacaacaataccagccttccaacttctgtaaataacggcggtacgccagtgccaccagtaccgttacctttcggtatacctcctttccccatgtttccaatgcccttcatgcctccaacggctactatcacaaatcctcatcaagctgacgcaagccctaagaaatgaataacaatactgacagtactaaataattgcctacttggcttcacatacgttgcatacgtcgatatagataataatgataatgacagcaggattatcgtaatacgtaatagttgaaaatctcaaaaatgtgtgggtcattacgtaaataatgataggaatgggattcttctatttttcctttttccattctagcagccgtcgggaaaacgtggcatcctctctttcgggctcaattggagtcacgctgccgtgagcatcctctctttccatatctaacaactgagcacgtaaccaatggaaaagcatgagcttagcgttgctccaaaaaagtattggatggttaataccatttgtctgttctcttctgactttgactcctcaaaaaaaaaaaatctacaatcaacagatcgcttcaattacgccctcacaaaaacttttttccttcttcttcgcccacgttaaattttatccctcatgttgtctaacggatttctgcacttgatttattataaaaagacaaagacataatacttctctatcaatttcagttattgttcttccttgcgttattcttctgttcttctttttcttttgtcatatataacCATAACCAAGTAATACATATTCAAAAGCCCTCTtACAATGCCATCCAAGTTGGccattacctctatgtctttgggtagatgttatgccggtcattctttcactactaagttggatatggctagaaagtacggttaccaaggtttggaattattccatgaagatttggctgatgtcgcctatagattgtctggtgaaactccatctccatgtggtccatcaccagctgctcaattgtctgctgctagacaaattttgagaatgtgccaagtcagaaacatcgaaatcgtttgcttgcaaccattctctcaatacgatggtttgttggatagagaagaacacgaaagaagattggaacaattggaattctggatcgaattggcccatgaattggataccgatattattcaaattccagccaacttcttgccagccgaagaagttactgaagatatctctttgattgtctccgacttgcaagaagtagctgatatgggtttacaagctaacccaccaattagattcgtttacgaagctttgtgttggtccactagagttgatacttgggaaagatcttgggaagttgttcaaagagttaacagaccaaacttcggtgtttgtttggacacttttaacattgccggtagagtttatgctgatccaactgttgcttctggtagaactccaaatgctgaagaagctatcagaaagtccattgccagattggttgaaagagttgacgtttccaaggttttctacgttcaagttgttgatgccgaaaagttgaagaaaccattggttccaggtcacagattctatgatccagaacaaccagctagaatgtcttggtctagaaactgcagattattctacggtgaaaaggatagaggtgcttacttgccagtaaaagaaattgcttgggcttttttcaacggtttgggttttgaaggttgggtttccttagaattattcaacagaagaatgtccgataccggttttggtgttccagaagaattagctagaagaggtgctgtttcttgggctaaattggttagagatatgaagatcaccgttgactctccaactcaacaacaagctacacaacaacctatcagaatgttgtCTTTGTCAGCTGCTTTGTGAACACAGGCtATAGATTAATTTAAACAGTATATGTACAGTTTtatatatatatatatatatatatacatatataaagaaacctgtgcgttttttgtattttcaaatacatttagttttggcgccacttcTATAAAAGGTCTCATGATATTGTTAC'
CCM004['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM004['Primer_up_U']='GV1' #user site up
CCM004['Primer_down_U']='U1'#user site down
CCM004['Primer_up']=''
CCM004['Primer_up_Tm']=''
CCM004['Primer_down']=''
CCM004['Primer_down_Tm']=''

CCM007={}
CCM007['name']='pCCM007(pTDH3:KpAroyB:tVPS13)'
CCM007['seq']='ATAAAAAACACGCTTTTTCAgttcgagtttatcattatcaatactgccatttcaaagaatacgtaaataattaatagtagtgattttcctaactttatttagtcaaaaaattagccttttaattctgctgtaacccgtacatgcccaaaatagggggcgggttacacagaatatataacatcgtaggtgtctgggtgaacagtttattcctggcatccactaaatataatggagcccgctttttaagctggcatccagaaaaaaaaagaatcccagcaccaaaatattgttttcttcaccaaccatcagttcataggtccattctcttagcgcaactacagagaacaggggcacaaacaggcaaaaaacgggcacaacctcaatggagtgatgcaacctgcctggagtaaatgatgacacaaggcaattgacccacgcatgtatctatctcattttcttacaccttctattaccttctgctctctctgatttggaaaaagctgaaaaaaaaggttgaaaccagttccctgaaattattcccctacttgactaataagtatataaagacggtaggtattgattgtaattctgtaaatctatttcttaaacttcttaaattctacttttatagttagtcttttttttagttttaaaacaccaagaacttagtttcgaataaacacacataaacaaacaaaagtgcaggtagtgcaggtaaaacaatgaagttgatcatcggtatgactggtgctacaggtgctccattgggtgttgctttgttgcaagctttgagagatatgccagaagttgaaacccatttggttatgtctaaatgggctaagaccaccattgaattggaaactccatggactgctagagaagttgctgctttggctgatttttctcattctccagctgatcaagctgctactatttcttctggttctttcagaactgatggtatgatcgttattccatgctctatgaaaaccttggctggtattagagctggttatgctgaaggtttggttggtagagctgctgatgttgttttgaaagaaggtagaaagttggtcttggtcccaagagaaatgccattgtctactatccatttggaaaacatgttggccttgtctagaatgggtgtagctatggttccaccaatgccagcttattacaatcatccagaaaccgttgatgacatcaccaaccatatagttaccagagttttggaccaattcggtttggattatcacaaagctagaagatggaacggtttgagaactgctgaacaattcgctcaagaaattgaatgaatCGCACGCATTCCGTTGACACAGGCtTCACATATGAAAGTATATACCCGcttttgtacactatgtagctataattcaatcgtattattgtagctccgcacgaccatgccttagaaataTCCGCAGCGCGC'
CCM007['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM007['Primer_up_U']='U1' #user site up
CCM007['Primer_down_U']='U2'#user site down
CCM007['Primer_up']=''
CCM007['Primer_up_Tm']=''
CCM007['Primer_down']=''
CCM007['Primer_down_Tm']=''

CCM008={}
CCM008['name']='pCCM008(pTEF1:Aroy.Ciso:tIDP1)'
CCM008['seq']='GCACACACCATAGCTTCAAaatgtttctactccttttttactcttccagattttctcggactccgcgcatcgccgtaccacttcaaaacacccaagcacagcatactaaatttcccctctttcttcctctagggtgtcgttaattacccgtactaaaggtttggaaaagaaaaaagagaccgcctcgtttctttttcttcgtcgaaaaaggcaataaaaatttttatcacgtttctttttcttgaaaatttttttttttgatttttttctctttcgatgacctcccattgatatttaagttaataaacggtcttcaatttctcaagtttcagtttcatttttcttgttctattacaactttttttacttcttgctcattagaaagaaagcatagcaatctaatctaagttttaattacaatctgtcataaaacaatgaccgccccaatccaagatttgagagatgctattgctttgttacaacaacacgacaatcaatacttggaaaccgatcatccagttgatccaaatgctgaattggctggtgtttacagacatattggtgctggtggtactgtaaaaagaccaactagaattggtccagccatgatgttcaacaacattaagggttatccacactccagaatcttggttggtatgcatgcttctagacaaagagcagctttgttgttgggttgtgaagcttctcaattggctttggaagttggtaaagctgttaagaaaccagttgctccagttgttgttccagcttcttctgctccatgtcaagaacaaattttcttggctgatgatccagacttcgatttgagaactttgttgccagctcataccaacactccaattgatgctggtccatttttttgtttgggtttggctttagcttctgatcctgttgatgcttctttgaccgatgttaccattcatagattgtgcgttcaaggtagagatgaattgtctatgtttttggctgccggtagacatatcgaagtttttagacaaaaagctgaagctgctggtaagccattgccaattactattaacatgggtttagatccagccatctacattggtgcttgttttgaagctccaactactccatttggttacaacgaattgggtgttgctggtgctttgagacaaagaccagttgaattggttcaaggtgtttctgttccagaaaaggctattgctagagccgaaatagttatcgaaggtgaattattgccaggtgtcagagttagagaagatcaacatacaaattccggtcatgctatgccagaatttccaggttattgtggtggtgctaatccatctttgccagttattaaggttaaggccgttaccatgagaaacaacgctattttacaaactttggtcggtccaggtgaagaacatacaactttggctggtttgccaaccgaagcttctatttggaatgctgttgaagctgcaattccaggtttcttgcaaaatgtttatgctcatacagctggtggtggtaagttcttgggtatattgcaagtcaagaaaagacaaccagctgacgaaggtagacaaggtcaagctgctttattagctttggctacttactccgaattgaagaatatcatcttggtcgatgaagatgttgatatcttcgattccgatgatattttgtgggctatgactactagaatgcaaggtgatgtttccattactaccattccaggtattagaggtcaccaattagatccatctcaaaccccagaatactccccatcaattagaggtaatggtatctcctgtaagaccattttcgattgcactgttccatgggctttgaagtctcattttgaaagagcaccatttgctgacgttgatcctagaccttttgctccagaatatttcgctagattggaaaagAATCAAGGTTCCGCTAAGTAAACACAGGCtTCGAATTTACGTAGCCCAATCTAccacttttttttttcattttttaaagtgttatacttagttatgctctaggataatgaactacttttttttttttttttttactgttatcataaatatatataccttattgttgtttgcaaccgtcggttaattccttatcaaggttccCCAAGTTCGGATCATTACCATC'
CCM008['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM008['Primer_up_U']='U2' #user site up
CCM008['Primer_down_U']='GV2'#user site down
CCM008['Primer_up']=''
CCM008['Primer_up_Tm']=''
CCM008['Primer_down']=''
CCM008['Primer_down_Tm']=''

pCCM012_13_14_15=[CCM004,CCM007,CCM008]

In [42]:
CCM009={}
CCM009['name']='pCCM009(pCCW12:AroYD:tPRM9)'
CCM009['seq']='AAAGAAACTTAATACGTTATGCCgtaatgaagggctaccaaaaacgataatctcaactgtaaacaggtacaatgcggacccttttgccacaaaacatacatcattcattgccggaaaaagaaagaagtgaagacagcagtgcagccagccatgttgcgccaatctaattatagatgctggtgccctgaggatgtatctggagccagccatggcatcatgcgctaccgccggatgtaaaatccgacacgcaaaagaaaaccttcgaggttgcgcacttcgcccacccatgaaccacacggttagtccaaaaggggcagttcagattccagatgcgggaattagcttgctgccaccctcacctcactaacgctgcggtgtgcggatacttcatgctatttatagacgcgcgtgtcggaatcagcacgcgcaagaaccaaatgggaaaatcggaatgggtccagaactgctttgagtgctggctattggcgtctgatttccgttttgggaatcctttgccgcgcgcccctctcaaaactccgcacaagtcccagaaagcgggaaagaaataaaacgccaccaaaaaaaaaaaaataaaagccaatcctcgaagcgtgggtggtaggccctggattatcccgtacaagtatttctcaggagtaaaaaaaccgtttgttttggaattccccatttcgcggccacctacgccgctatctttgcaacaactatctgcgataactcagcaaattttgcatattcgtgttgcagtattgcgataatgggagtcttacttccaacataacggcagaaagaaatgtgagaaaattttgcatcctttgcctccgttcaagtatataaagtcggcatgcttgataatctttctttccatcctacattgttctaattattcttattctcctttattctttcctaacataccaagaaattaatcttctgTCATTCGCTTAAACACTATATCAATAAGCCCTCTtACAATGATCTGTCCAAGATGCgccgacgaaaaaattgaagttatggctacttctccagttaagggtgtttggactgtttatcaatgtcaacactgcttgtacacttggagagatactgaaccattgagaagaacctctagagaacattaccctgaagctttcagaatgacccaaaaggatattgatgaagctccacaagttcctcatgttccacCATTATTGCCAGAAGATAAGAGATAAACACAGGCtACAGAAGACGGGAGACACtagcacacaactttaccaggcaaggtatttgacgctagcatgtgtccaattcagtgtcatttatgattttttgtagtaggatataaatatatacagcgctccaaatagtgcggttgccccaaaaacaccacggaacctcatctgttctcgtactttgttgtgacaaagtagctcactgccttattatcacattttcattatgcaacgctTCGGAAAATACGATGTTGAAAAT'
CCM009['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM009['Primer_up_U']='GV1' #user site up
CCM009['Primer_down_U']='U1'#user site down
CCM009['Primer_up']=''
CCM009['Primer_up_Tm']=''
CCM009['Primer_down']=''
CCM009['Primer_down_Tm']=''

CCM005={}
CCM005['name']='pCCM005(pTPI1:NrsR:LSC2t)'
CCM005['seq']='tAAGGATGAGCCAAGAATAAGGgaacaagattttggtaatttccaaaaaatcaatagcatgcaggacgttatgaagaagagatctacgtatggtcatttcttcttcagattccctcatggagaaagtgcggcagatgtatatgacagagtcgccagtttccaagagactttattcaggcacttccatgataggcaagagagaagacccagagatgttgttgtcctagttacacatggtatttattccagagtattcctgatgaaatggtttagatggacatacgaagagtttgaatcgtttaccaatgttcctaacgggagcgtaatggtgatggaactggacgaatccatcaatagatacgtcctgaggaccgtgctacccaaatggactgattgtgagggagacctaactacatagtgtttaaagattacggatatttaacttacttagaataatgccatttttttgagttataataatcctacgttagtgtgagcgggatttaaactgtgaggaccttaatacattcagacacttctgcggtatcaccctacttattcccttcgagattatatctaggaacccatcaggttggtggaagattacccgttctaagacttttcagcttcctctattgatgttacacctggacaccccttttctggcatccagtttttaatcttcagtggcatgtgagattctccgaaattaattaaagcaatcacacaattctctcggataccacctcggttgaaactgacaggtggtttgttacgcatgctaatgcaaaggagcctatatacctttggctcggctgctgtaacagggaatataaagggcagcataatttaggagtttagtgaacttgcaacatttactattttcccttcttacgtaaatatttttctttttaattctaaatcaatctttttcaattttttgtttgtattcttttcttgcttaaatctataACTACAAAAAACACATACATAAACTAAAAAGCCCTCTtACCATGGGTACCACTCTtgacgacacggcttaccggtaccgcaccagtgtcccgggggacgccgaggccatcgaggcactggatgggtccttcaccaccgacaccgttttccgcgtcaccgccaccggggacggcttcaccctgcgggaggtgccggtggacccgcccctgaccaaggtgttccccgacgacgaatcggacgacgaatcggacgacggggaggacggcgacccggactcccggacgttcgtcgcgtacggggacgacggcgacctggcgggcttcgtggtcgtctcgtactccggctggaaccgccggctgaccgtcgaggacatcgaggtcgccccggagcaccgggggcacggggtcgggcgcgcgttgatggggctcgcgacggagttcgcccgcgagcggggcgccgggcacctctggctggaggtcaccaacgtcaacgcaccggcgatccacgcgtaccggcggatggggttcaccctctgcggcctggacaccgccctgtacgacggcaccgcctcggacggcgagcaggcgctctacatgagcaTGCCCTGCCCCTAAACACAGGCtGCTTCTCGAGAAAAACAAAAGAGTTAataataaagtatatatgcttttttactattaataaagtttcttattttccctacatgtatacatacatataaatatactcttttGGAAAAAATTTCTTTTTTTTTTTTTTAATTTT'
CCM005['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM005['Primer_up_U']='U1' #user site up
CCM005['Primer_down_U']='U2'#user site down
CCM005['Primer_up']=''
CCM005['Primer_up_Tm']=''
CCM005['Primer_down']=''
CCM005['Primer_down_Tm']=''

CCM006={}
CCM006['name']='pCCM006(pCyc1:CaCatA:CPS1t)'
CCM006['seq']='tCAGCATTTTCAAAGGTGTgttcttcgtcagacatgttttagtgtgtgaatgaaataggtgtatgttttctttttgctagacaataattaggaacaaggtaagggaactaaagtgtagaataagattaaaaaagaagaacaagttgaaaaggcaagttgaaatttcaagaaaaaagtcaattgaagtacagtaaattgacctgaatatatctgagttccgacaacaatgagtttaccaaagagaacaatggaataggaaactttgaacgaagaaaggaaagcaggaaaggaaaaaatttttaggctcgagaacaatagggcgaaaaaacaggcaacgaacgaacaatggaaaaacgaaaaaaaaaaaaaaaaacacagaaaagaatgcagaaagatgtcaactgaaaaaaaaaaaggtgaacacaggaaaaaaaataaaaaaaaaaaaaaaaaaaggaggacgaaacaaaaaagtgaaaaaaaatgaaaatttttttggaaaaccaagaaatgaattatatttccgtgtgagacgacatcgtcgaatatgattcagggtaacagtattgatgtaatcaatttcctacctgaatctaaaattcccgggagcaagatcaagatgttttcaccgatctttccggtctctttggccggggtttacggacgatggcagaagaccaaagcgccagttcatttggcgagcgttggttggtggatcaagcccacgcgtaggcaatcctcgagcagatccgccaggcgtgtatatatagcgtggatggccaggcaactttagtgctgacacatacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaAACACAAATACACACACTAAATTAATAAGCCCTCTtACAATGTCCCAAGCTTTCAccgaatctgttaagacttctttgggtccaaatgctactccaagagctaaaaagttgattgcctctttggttcaacacgttcatgatttcgctagagaaaaccatttgactaccgaagattggttgtggggtgttgatttcattaacagaattggtcaaatgtccgactccagaagaaacgaaggtattttggtttgcgatatcatcggtttggaaaccttggttgatgctttgactaacgaatccgaacaatctaaccatacctcctctgctattttgggtcctttttacttgccagattctccagtttatccaaacggtggttctatcgttcaaaaggctattccaactgatgttaagtgcttcgttagaggtaaggttactgatactgaaggtaaaccattgggtggtgctcaattggaagtttggcaatgtaattctgctggtttctactctcaacaagctgatcatgatggtccagaattcaatttgagaggtactttcattaccgacgacgaaggtaattactccttcgaatgtttaagaccaacctcctatccaattccatacgatggtcctgctggtgatttgttgaaaatcatggatagacatccaaacagaccatcccatattcattggagagtttctcatccaggttaccatactttgatcacccaaatctatgatgctgaatgtccatacaccaacaacgattctgtttacgctgttaaggatgacatcatcgttcacttcgaaaaggttgataacaaggataaggatttggtcggtaaggtcgaatacaagttggattacgatatttccttggccaccgaatcctctattcaagaagctagagctgctgctaaagctagaCAAGATGCTGAAATCAAGTTGTAAACACAGGCtGCGCAATGATTGAATAGTCAAAgattttttttttttaattttttttttttaattttttttttttttcatagaactttttatttaaataaatcacgtctatatatgtatcagtataacgtaaaaaaaaaaacaccgtcagttaaacaaaacataaataaaaaaaaAAAGAAGTGTCAAATCAAGTGTCAAATC'
CCM006['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM006['Primer_up_U']='U2' #user site up
CCM006['Primer_down_U']='U3'#user site down
CCM006['Primer_up']=''
CCM006['Primer_up_Tm']=''
CCM006['Primer_down']=''
CCM006['Primer_down_Tm']=''

CCM003={}
CCM003['name']='pCCM003(pTDH2:Tkl1:TDH3t)'
CCM003['seq']='ATCTAGATCAGAGGGTGGTaaatgaagtgtaatagtattcatttttcttataaatcatcccttccgtgatttatacaaaagaagaggagaatatgctgaatacttggtatattactctacattatactcttatcttgacgggtattctgagcatcttactcagtttcaagatcttttaatgtccaaaaacatttgagccgatctaaatacttctgtgttttcattaatttataaattgtactcttttaagacatggaaagtaccaacatcggttgaaacagtttttcatttacttatggtttattggtttttccagtgaatgattatttgtcgttaccctttcgtaaaagttcaaacacgtttttaagtattgtttagttgctctttcgacatatatgattatccctgcgcggctaaagttaaggatgcaaaaaacataagacaactgaagttaatttacgtcaattaagttttccagggtaatgatgttttgggcttccactaattcaataagtatgtcatgaaatacgttgtgaagagcatccagaaataatgaaaagaaacaacgaaactgggtcggcctgttgtttcttttctttaccacgtgatctgcggcatttacaggaagtcgcgcgttttgcgcagttgttgcaacgcagctacggctaacaaagcctagtggaactcgactgatgtgttagggcctaaaactggtggtgacagctgaagtgaactattcaatccaatcatgtcatggctgtcacaaagaccttgcggaccgcacgtacgaacacatacgtatgctaatatgtgttttgatagtacccagtgatcgcagacctgcaatttttttgtaggtttggaagaatatataaaggttgcactcattcaagatagtttttttcttgtgtgtctattcattttattattgtttgtttaaatgttaaaaaaaccaagaacttagtttcaaaTTAAATTCATCACACAAACAAACAAAACAAAAGCCCTCTtACAATGACTCAATTCACTGACATTGATaagctagccgtctccaccataagaattttggctgtggacaccgtatccaaggccaactcaggtcacccaggtgctccattgggtatggcaccagctgcacacgttctatggagtcaaatgcgcatgaacccaaccaacccagactggatcaacagagatagatttgtcttgtctaacggtcacgcggtcgctttgttgtattctatgctacatttgactggttacgatctgtctattgaagacttgaaacagttcagacagttgggttccagaacaccaggtcatcctgaatttgagttgccaggtgttgaagttactaccggtccattaggtcaaggtatctccaacgctgttggtatggccatggctcaagctaacctggctgccacttacaacaagccgggctttaccttgtctgacaactacacctatgttttcttgggtgacggttgtttgcaagaaggtatttcttcagaagcttcctccttggctggtcatttgaaattgggtaacttgattgccatctacgatgacaacaagatcactatcgatggtgctaccagtatctcattcgatgaagatgttgctaagagatacgaagcctacggttgggaagttttgtacgtagaaaatggtaacgaagatctagccggtattgccaaggctattgctcaagctaagttatccaaggacaaaccaactttgatcaaaatgaccacaaccattggttacggttccttgcatgccggctctcactctgtgcacggtgccccattgaaagcagatgatgttaaacaactaaagagcaaattcggtttcaacccagacaagtcctttgttgttccacaagaagtttacgaccactaccaaaagacaattttaaagccaggtgtcgaagccaacaacaagtggaacaagttgttcagcgaataccaaaagaaattcccagaattaggtgctgaattggctagaagattgagcggccaactacccgcaaattgggaatctaagttgccaacttacaccgccaaggactctgccgtggccactagaaaattatcagaaactgttcttgaggatgtttacaatcaattgccagagttgattggtggttctgccgatttaacaccttctaacttgaccagatggaaggaagcccttgacttccaacctccttcttccggttcaggtaactactctggtagatacattaggtacggtattagagaacacgctatgggtgccataatgaacggtatttcagctttcggtgccaactacaaaccatacggtggtactttcttgaacttcgtttcttatgctgctggtgccgttagattgtccgctttgtctggccacccagttatttgggttgctacacatgactctatcggtgtcggtgaagatggtccaacacatcaacctattgaaactttagcacacttcagatccctaccaaacattcaagtttggagaccagctgatggtaacgaagtttctgccgcctacaagaactctttagaatccaagcatactccaagtatcattgctttgtccagacaaaacttgccacaattggaaggtagctctattgaaagcgcttctaagggtggttacgtactacaagatgttgctaacccagatattattttagtggctactggttccgaagtgtctttgagtgttgaagctgctaagactttggccgcaaagaacatcaaggctcgtgttgtttctctaccagatttcttcacttttgacaaacaacccctagaatacagactatcagtcttaccagacaacgttccaatcatgtctgttgaagttttggctaccacatgttggggcaaatacgctcatcaatccttcggtattgacagatttggtgcctccggtaaggcaccagaagtcttcaagttcttcggtttcaccccagaaggtgttgctgaaagagctcaaaagaccattgcattctataagggtgacaagctaatttcTCCTTTGAAAAAAGCTTTCTGAACACAGGCtGTGAATTTACTTTAAATCTTGCATTTAAATAAATTTTCTTTTTATAGCTTTATGACTTAGTTTCAATTTATATACTATTTTAATGACATTTTCGATTCATTGATTGAAAGCTTTGTGTTTTTTCTTGATGCGCTATTGCATTGTTCTTGTCTTTTTCGCCACATGTAATATCTGTAGTAGATACCTGATACATTGTGGATGCTGAGTGAAATTTTAGTTAATAATGGAGGCGCTCTTAATAATTTTGGGGATATTGGCTTTTTTTTTTAAAGTTTACAAATGAATTTTTTCCGCCAGGATAACGATTCTGAAGTTACTCTTAGCGTTCCTATCGGTACAGCCATCAAATCATGCCTATAAATCATGCCTATATTTGCGTGCAGTCAGTATCATCTACATGAAAAAAACTCCCGCAATTTCTTATAGAATACGTTGAAAATTAAATGTACGCGCCAAGATAAGATAACATATATCTAGATGCAGTAATATACACAGATTCCCGCGGACGTGGGAAGGAAAAAATTAGATAACAAAATCTGAGTGATATGGAAATTCCGCTGTATAGCTCATATCTTTCCCTACCTGCAGA'
CCM003['need_reverse']=0#if parts needs to be reversed:1 else:0
CCM003['Primer_up_U']='U3' #user site up
CCM003['Primer_down_U']='GV2'#user site down
CCM003['Primer_up']=''
CCM003['Primer_up_Tm']=''
CCM003['Primer_down']=''
CCM003['Primer_down_Tm']=''
pCCM016=[CCM009,CCM005,CCM006,CCM003]


In [1]:
CCM001={}
CCM001['name']='pCCM001(pCYC_benO:yEGFP:Cyc1t)'
CCM001['seq']='GATCCAGGCAACTTTAGTGCtgacacataatactccataggtattttattatacaaataatgtgtttgaacttattaaaacattcttttaaggtataaacaacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaataatctgtcataaaacaatgtctaaaggtgaagaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaaggtgatgctacttacggtaaattgaccttaaaatttatttgtactactggtaaattgccagttccatggccaaccttagtcactactttcggttatggtgttcaatgttttgcgagatacccagatcatatgaaacaacatgactttttcaagtctgccatgccagaaggttatgttcaagaaagaactatttttttcaaagatgacggtaactacaagaccagagctgaagtcaagtttgaaggtgataccttagttaatagaatcgaattaaaaggtattgattttaaagaagatggtaacattttaggtcacaaattggaatacaactataactctcacaatgtttacatcatggctgacaaacaaaagaatggtatcaaagttaacttcaaaattagacacaacattgaagatggttctgttcaattagctgaccattatcaacaaaatactccaattggtgatggtccagtcttgttaccagacaaccattacttatccactcaatctgccttatccaaagatccaaacgaaaagagagaccacatggtcttgttagaatttgttactgctgctggtattacccatggtatggatgaattgtacaaataaatcgcgtgcattcatccgctctaaccgaaaaggaaggagttagacaacctgaagtctaggtccctatttatttttttatagttatgttagtattaagaacgttatttatatttcaaatttttcttttttttctgtacagacgcgtgtacgcatgtaacattatactgaaaaccttgcttgagaaggttttgggacgctcgaAGATCGCGTCAGCTGAA'
CCM001['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM001['Primer_up_U']='GV1' #user site up
CCM001['Primer_down_U']='U1'#user site down
CCM001['Primer_up']=''
CCM001['Primer_up_Tm']=''
CCM001['Primer_down']=''
CCM001['Primer_down_Tm']=''

#this part also works for pCCM010
CCM002={}
CCM002['name']='pCCM002(tADH1::BenM::pRev1)'
CCM002['seq']='gcgacctcatgctatacctgagaaagcaacctgacctacaggaaagagttactcaagaataagaattttcgttttaaaacctaagagtcactttaaaatttgtatacacttattttttttataacttatttaataataaaaatcataaatcataagaaattcgcttatttagaagtgtcaacaacgtatctaccaacggaatgCGTGCGAtGAATGCGGCCGCTTtaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaagagctccagcttttgttccctttagtgagggttaattgcgcgcttggcgtaatcatggtcatagctgtttccTGTGTGAAATTGTTATCCGCT'
CCM002['need_reverse']=1 #if parts needs to be reversed:1 else:0
CCM002['Primer_up_U']='U1' #user site up
CCM002['Primer_down_U']='GV2'#user site down
CCM002['Primer_up']=''
CCM002['Primer_up_Tm']=''
CCM002['Primer_down']=''
CCM002['Primer_down_Tm']=''

pCCM017_18_19_20=[CCM001,CCM002]

### Design

In [10]:

Parts =User_Assebmly(pCCM011, 50)
print(Parts)

Will use input Tm_target for design:  50
[{'name': 'pCfB1241_TDH3p', 'seq': 'TTTGTTTGTTTATGTGTGTTTATTCGAAACTAAGTTCTTGGTGTTTTAAAACTAAAAAAAAGACTAACTATAAAAGTAGAATTTAAGAAGTTTAAGAAATAGATTTACAGAATTACAATCAATACCTACCGTCTTTATATACTTATTAGTCAAGTAGGGGAATAATTTCAGGGAACTGGTTTCAACCTTTTTTTTCAGCTTTTTCCAAATCAGAGAGAGCAGAAGGTAATAGAAGGTGTAAGAAAATGAGATAGATACATGCGTGGGTCAATTGCCTTGTGTCATCATTTACTCCAGGCAGGTTGCATCACTCCATTGAGGTTGTGCCCGTTTTTTGCCTGTTTGTGCCCCTGTTCTCTGTAGTTGCGCTAAGAGAATGGACCTATGAACTGATGGTTGGTGAAGAAAACAATATTTTGGTGCTGGGATTCTTTTTTTTTCTGGATGCCAGCTTAAAAAGCGGGCTCCATTATATTTAGTGGATGCCAGGAATAAACTGTTCACCCAGACACCTACGATGTTATATATTCTGTGTAACCCGCCCCCTATTTTGGGCATGTACGGGTTACAGCAGAATTAAAAGGCTAATTTTTTGACTAAATAAAGTTAGGAAAATCACTACTATTAATTATTTACGTATTCTTTGAAATGGCAGTATTGATAATGATAAACTCGAACTGAAAAAGCGTGTTTTTTATAGC', 'need_reverse': 1, 'Primer_up_U': 'GV1', 'Primer_down_U': 'U7', 'Primer_up': 'ACCTGCACuTTTGTTTGTTTATGTGTGTTTATTCG', 'Primer_up_Tm': 51.22163584953182, 'Primer_down': 'CGTGCGAuGCTATAAAAAACACGCTTTTTCA', 'Primer_down_Tm': 

#### compile into a table

In [11]:



primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]
i=1
for part in Parts:
    #here we adapt the order with which the overhang names are appended to match the Overhang names to the inversion of sequences
    if part['need_reverse']==0:
        primer_OH.append(part['Primer_up' +'_U'])
        primer_OH.append(part['Primer_down'+'_U'])
    elif part['need_reverse']==1:
        primer_OH.append(part['Primer_down'+'_U'])
        primer_OH.append(part['Primer_up' +'_U'])
        
    for primer in ['Primer_up', 'Primer_down']:
        primer_name.append('CCM.0')#.zfill fills primer number wiht leading 0
        primer_seq.append(part[primer])
        primer_tm.append(round(part[primer +'_Tm'],2))
        primer_template_len.append(len(part['seq']))
        primer_template_name.append(part['name'])
        #primer_OH.append(part[primer +'_U'])
        primer_dir.append('fwd'  if primer[7:] == 'up' else 'rvs' )
        benchling.append(part[primer].replace('u','t'))
        i+=1
## need to fix the current overhang name bug (if seq needs inversion the OH name doenst match)
        


df = np.stack([primer_name, primer_seq,primer_dir, primer_tm,primer_OH, primer_template_name, primer_template_len, benchling] ).T
Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Table.set_index('Primer Name')

,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
Primer Name,,,,,,,
CCM.0,ACCTGCACuTTTGTTTGTTTATGTGTGTTTATTCG,fwd,51.22,U7,pCfB1241_TDH3p,701,ACCTGCACtTTTGTTTGTTTATGTGTGTTTATTCG
CCM.0,CGTGCGAuGCTATAAAAAACACGCTTTTTCA,rvs,50.16,GV1,pCfB1241_TDH3p,701,CGTGCGAtGCTATAAAAAACACGCTTTTTCA
CCM.0,AGTGCAGGuAAACAATGAAGTTGATCATCGGT,fwd,51.56,U7,KpAroY.B-P146T.gBlock,599,AGTGCAGGtAAACAATGAAGTTGATCATCGGT
CCM.0,AGCCTGTGuTCATTCAATTTCTTGAGCGAATTG,rvs,50.87,U2,KpAroY.B-P146T.gBlock,599,AGCCTGTGtTCATTCAATTTCTTGAGCGAATTG
CCM.0,CACGCGAuGCGCGCTGCGGA,fwd,51.74,GV2,VPS13t,104,CACGCGAtGCGCGCTGCGGA
CCM.0,ACACAGGCuTCACATATGAAAGTATATACCCG,rvs,50.63,U2,VPS13t,104,ACACAGGCtTCACATATGAAAGTATATACCCG
